# Lab #3 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/basic-operations-workshop/blob/main/lab3.ipynb)
1. Install dependencies
2. Create a pinecone index 
3. Load public image dataset(fashion-mnist) and create vector embeddings from the dataset
4. Create a local parquet backup of your image embeddings
5. Insert the fashion-mnist embeddings into Pinecone
6. Run a nearest neighbor search on a sample image that is not in the training dataset
7. Run a nearest neighbor search on 100 random test images that are not in the training dataset
8. Run a load test script to simulate 10 concurrent users querying the index
9. TEARDOWN: Delete the index 

# 1. Install Pinecone client 
Use the following shell command to install Pinecone:

In [1]:
!pip install -U "pinecone-client[grpc]" "python-dotenv" "torch" "torchvision" "pillow" "ftfy" "regex" "git+https://github.com/openai/clip.git" "datasets" "locust"

try:
    import pinecone
    import dotenv
    import numpy
    import torch
    import clip
    import datasets
    print("SUCCESS: lab dependencies are installed.")
except ImportError as ie:
    print(f"ERROR: key deendencies are not installed: {ie}")

  Cloning https://github.com/openai/clip.git to /private/var/folders/n7/j7krsnmx3wl7_bjrwhx2z7ym0000gn/T/pip-req-build-ob81lah7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/clip.git /private/var/folders/n7/j7krsnmx3wl7_bjrwhx2z7ym0000gn/T/pip-req-build-ob81lah7
  Resolved https://github.com/openai/clip.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


SUCCESS: lab dependencies are installed.


# 2. Create a pinecone index 
We will create an index that will be used to load/query a hugging face dataset.

In [4]:
from dotenv import load_dotenv
import os
import pinecone

load_dotenv('.env')

PINECONE_INDEX_NAME = os.environ['PINECONE_INDEX_NAME']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_ENVIRONMENT = os.environ['PINECONE_ENVIRONMENT']
METRIC = os.environ['METRIC']
DIMENSIONS = int(os.environ['DIMENSIONS'])
INDEX_NAMESPACE = os.environ['INDEX_NAMESPACE']

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

if (PINECONE_INDEX_NAME in pinecone.list_indexes()) != True:  
    pinecone.create_index(PINECONE_INDEX_NAME, dimension=DIMENSIONS, metric=METRIC, pods=1, replicas=1, pod_type="s1.x1")
else:
    print(f"Index {PINECONE_INDEX_NAME} already exists")

print(f"Index Description: {pinecone.describe_index(name=PINECONE_INDEX_NAME)}")

Index Description: IndexDescription(name='james-williams', metric='euclidean', replicas=1, dimension=512.0, shards=1, pods=1, pod_type='s1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')


# 3. Load public image dataset(fashion-mnist) and create vector embeddings from the dataset

Use the following shell command to download the [fashion-mnist](https://huggingface.co/datasets/fashion_mnist) training dataset from Hugging Face so that we can create vector embeddings that uses a label(image class) as meta-data from this dataset. The meta-data labels mappings are:

| Label  | Description |
| ------ | ----------- |
| 0      | T-shirt/top |
| 1      | Trouser     |
| 2      | Pullover    |
| 3      | Dress       |
| 4      | Coat        |
| 5      | Sandal      |
| 6      | Shirt       |
| 7      | Sneaker     |
| 8      | Bag         |
| 9      | Ankle boot  |

The Fashion-MNIST dataset is a dataset of Zalando's article images, consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

The accuracy you can achieve depends on the model and the preprocessing steps you use. Here's a rough guideline for what you might expect with some classic machine learning algorithms:

1. **Random Forest:** Around 85-89% accuracy.
2. **Support Vector Machines (SVM):** Around 85-90% accuracy, depending on kernel and hyperparameters.
3. **k-Nearest Neighbors (k-NN):** Around 85-88% accuracy.
4. **Logistic Regression:** Around 82-85% accuracy.
5. **Gradient Boosting Machines (e.g., XGBoost):** Around 87-90% accuracy.

Keep in mind these numbers are approximate and can vary based on the exact preprocessing, feature extraction, and hyperparameter tuning you do. In general, deep learning models, especially Convolutional Neural Networks (CNNs), tend to perform better on image classification tasks like Fashion-MNIST, potentially reaching over 90-95% accuracy.

But for classic machine learning models, anything in the 85-90% range can be considered a reasonable result for the Fashion-MNIST dataset. It reflects a model that has learned something meaningful from the data but isn't necessarily state-of-the-art for this particular task.

In [5]:
from datasets import load_dataset
from tqdm.auto import tqdm  # progress bar
from PIL import Image
import torch
import clip
import time
import numpy as np

#  Load the fashion-mnist dataset - only retrieve 6K random images (10% of total dataset)
dataset = load_dataset("fashion_mnist")['train'].shuffle(seed=42).select(range(0,6000))
#dataset = load_dataset("fashion_mnist")['train']

label_descriptions = {0: "T-shirt/top", 
           1: "Trouser",
           2: "Pullover",
           3: "Dress",
           4: "Coat",
           5: "Sandal",
           6: "Shirt",
           7: "Sneaker",
           8: "Bag",
           9: "Ankle boot"}

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)
   
# Generate vector embeddings for each image in the dataset
id = 0
vectors = []
for img in tqdm(dataset, total=dataset.num_rows, desc='Images', position=0):
    with torch.no_grad():
        id += 1
        image_pp = preprocess(img['image']).unsqueeze(0).to(device)
        image_features = model.encode_image(image_pp)
        embedding = image_features.cpu().numpy().squeeze().tolist()
        meta_data = {"description": label_descriptions[img["label"]], "timestamp": time.time()}
        vectors.append({'id': str(id),
                        'values': embedding,
                        'metadata': meta_data})

Images: 100%|██████████| 6000/6000 [03:34<00:00, 27.91it/s]


# 4. Create a local parquet backup of your image embeddings

This is good practice because generating embeddings can be expensive and time consuming when calling hosted models like OpenAI. As you can see, even locally generated embeddings are time consuming.

In [17]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from IPython.display import display

df = pd.DataFrame(vectors)

#dipslay the first 5 rows of the dataframe
pd.set_option('display.max_colwidth', None)
display(df.head())

df.to_parquet('fashion-mnist-clip.parquet')

,id,values,metadata
0,1,"[0.008759277872741222, -0.20710694789886475, -0.3265663683414459, 0.05358453840017319, 0.34647610783576965, -0.3348323106765747, 0.16168484091758728, 0.9060096740722656, 0.3548886775970459, -0.04384024068713188, 0.08166541904211044, 0.2393338978290558, 0.6599251627922058, -0.32549598813056946, -0.2640305757522583, -0.072362519800663, 0.4160095453262329, 0.2451312243938446, 0.04245516657829285, 0.3809719681739807, -0.5581574440002441, 0.4062442183494568, 0.35916459560394287, -0.026508750393986702, -0.2544264793395996, -0.14446818828582764, -0.26534393429756165, -0.028064025565981865, 0.13279257714748383, -0.31222209334373474, 0.026539580896496773, -0.07840023189783096, 0.17045815289020538, -0.07644832134246826, -0.4124664068222046, -0.6990364193916321, -0.07593540847301483, 0.054352905601263046, -0.5394606590270996, -0.3521934747695923, -0.6636286973953247, -0.19592992961406708, -0.27161911129951477, -0.2148979753255844, -0.2216959297657013, 0.4042796194553375, -0.25046131014823914, 0.07209423184394836, 0.3029167950153351, -0.2142917960882187, 0.29765626788139343, 0.2673562467098236, 0.36764630675315857, -0.12876002490520477, -0.06875656545162201, 0.04690822958946228, 0.34548094868659973, -0.22725483775138855, -0.23107187449932098, -0.08655727654695511, 0.35345324873924255, -0.06281877309083939, -0.05350608006119728, -0.5631681084632874, 0.03890611603856087, 0.20459145307540894, 0.03106505237519741, -0.4141904413700104, -0.11418329179286957, -0.17679740488529205, -0.15880605578422546, 0.011449756100773811, 0.6305040717124939, -0.19833338260650635, -0.2886725068092346, 0.2577667534351349, 0.1635541468858719, 0.01885959319770336, 0.008384674787521362, -0.6606494188308716, -0.1398177146911621, -0.14677472412586212, 0.3102433383464813, 0.10178500413894653, 0.311718612909317, 0.19127772748470306, -0.19770747423171997, 0.061263423413038254, -0.29144954681396484, -0.2858257293701172, 0.2098928987979889, -0.3228234350681305, -8.724929809570312, 0.23969081044197083, 0.46594494581222534, 0.094818115234375, -0.09828225523233414, -0.08568913489580154, -0.11303391307592392, -0.43053391575813293, ...]","{'description': 'Dress', 'timestamp': 1691504808.082121}"
1,2,"[0.18876351416110992, 0.06803330779075623, -0.3696630001068115, -0.14380639791488647, -0.02326127700507641, 0.13572858273983002, 0.10369477421045303, 0.8926424384117126, 0.33766523003578186, 0.32612326741218567, 0.02722463198006153, 0.23189237713813782, 0.3104938566684723, -0.2723168432712555, -0.4215393662452698, 0.031867191195487976, 0.6063439846038818, 0.3050440847873688, 0.05424952134490013, -0.0072557092644274235, -0.5087708830833435, 0.4959316849708557, 0.0006730547174811363, -0.08289677649736404, -0.08398040384054184, 0.186918705701828, 0.04200270399451256, -0.1423955261707306, 0.22204463183879852, -0.040548380464315414, 0.26215046644210815, -0.40756309032440186, -0.09166142344474792, 0.16716548800468445, -0.3270414471626282, -0.7162606120109558, 0.11883895844221115, -0.038985736668109894, -0.2603166699409485, -0.4541040360927582, -0.6146935224533081, -0.44350990653038025, -0.28967905044555664, -0.32448065280914307, -0.21425308287143707, 1.066301703453064, -0.04915701597929001, 0.32419881224632263, 0.16908292472362518, -0.31009650230407715, 0.13615752756595612, 0.2629549503326416, 0.2929893732070923, 0.09002819657325745, 0.11788462847471237, 0.08777409046888351, 0.446664035320282, -0.13061974942684174, -0.3295435607433319, -0.06152321770787239, 0.6601959466934204, -0.0014343899674713612, -0.06366293877363205, -0.5479024648666382, -0.1107802614569664, 0.0633939728140831, -0.1153319701552391, -0.10133379697799683, -0.1450570672750473, -0.1259893774986267, -0.13065451383590698, -0.06449749320745468, 0.5432891845703125, -0.23775529861450195, -0.3399951756000519, 0.028745664283633232, 0.30851590633392334, 0.048755671828985214, -0.005466883536428213, -0.39040830731391907, -0.05489792302250862, -0.08486843854188919, 0.17055505514144897, -0.0897371098399162

# 5. Insert the fashion-mnist embeddings into Pinecone

The best way to do bulk updates is by batching the dataset. We will also use a namespace for the data. 

In [ ]:
from tqdm.auto import tqdm  # progress bar
import pinecone
import itertools

# Read Parquet file into a DataFrame
df = pd.read_parquet('fashion-mnist-clip.parquet')
df['values'] = df['values'].apply(lambda x: x.tolist())

# Convert DataFrame to a list of dictionaries
data_list = df.to_dict(orient='records')

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

index = pinecone.Index(PINECONE_INDEX_NAME)

# Obtain the upsert embeddings in batches of 100
batch_size = 100
id = 0
for vector_batch in tqdm(chunks(data_list, batch_size=batch_size), total=(len(vectors) / batch_size)):
   index.upsert(vector_batch, namespace=INDEX_NAMESPACE)

# 6. Run a nearest neighbor search on a sample image that is not in the training dataset

Download a sneaker image file from github that we will use to run a query to see if pinecone search returns the correct description "Sneaker". A query returns the correct result if the most common top_k description matches the test image.

You can change the top_k from 10 to 1 to 20 to see if the ANN results vary.

In [ ]:
import pinecone
from PIL import Image
import torch
import clip
import requests

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device) 

def image_to_embedding():
    
    url = "https://github.com/pinecone-io/basic-operations-workshop/blob/main/sneaker.jpeg?raw=true"
    response = requests.get(url)
    with open("sneaker.jpeg", "wb") as file:
      file.write(response.content)
    image_pp = preprocess(Image.open("./sneaker.jpeg")).unsqueeze(0).to(device)
    with torch.no_grad():
      embedding = model.encode_image(image_pp).squeeze().tolist()
    
    return embedding

index = pinecone.Index(PINECONE_INDEX_NAME)
top_k = 10

query_result = index.query(
  vector = image_to_embedding(),
  namespace=INDEX_NAMESPACE,
  top_k=top_k,
  include_values=False,
  include_metadata=True
)

top_k_success = False
match_cnt = 0
miss_categories = set()

my_list = query_result.matches
descriptions = [entry['metadata']['description'] for entry in my_list]
most_common_item = max(set(descriptions), key=descriptions.count)

if most_common_item == "Sneaker":
      top_k_success = True

for match in query_result.matches:
  if match.metadata['description'] == "Sneaker":
    match_cnt += 1
    top_k_contains = True
  else:
    miss_categories.add(match.metadata['description'])

print(f"Most common item matching result: {top_k_success}")
print(f"top_k: {top_k} match percentage is: {match_cnt/top_k * 100}%")
print(f"Match miss categories: {miss_categories} exepected 'Sneaker'")

# 7. Run a nearest neighbor search on 100 random test images that are not in the training dataset

Select 100 random test images. Keep in mind the model was NOT trained against these images. Obtain the percentage of pinecone queries that return the correct result in top_k. Feel free to play around with the "top_k" setting to see if you can increase the hit percentage. 

A query returns the correct result if the most common top_k description matches the test image.

In [ ]:
import clip
import torch
import pinecone
from datasets import load_dataset
from tqdm.auto import tqdm 
from collections import Counter

test_dataset = load_dataset("fashion_mnist")['test'].shuffle().select(range(0, 100))
#test_dataset = load_dataset("fashion_mnist")['test']

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device) 

label_descriptions = {0: "T-shirt/top", 
           1: "Trouser",
           2: "Pullover",
           3: "Dress",
           4: "Coat",
           5: "Sandal",
           6: "Shirt",
           7: "Sneaker",
           8: "Bag",
           9: "Ankle boot"}

# Generate vector embeddings for each image in the dataset
test_vectors = []
for img in tqdm(test_dataset, total=test_dataset.num_rows):
  image_pp = preprocess(img['image']).unsqueeze(0).to(device)
  embedding = model.encode_image(image_pp).squeeze().tolist()
    
  test_vectors.append({'embedding': embedding,
                        'description': label_descriptions[img["label"]]})
    
index = pinecone.Index(PINECONE_INDEX_NAME)
top_k = 10
top_k_success_cnt = 0

for v in test_vectors:

  top_k_success = False

  query_result = index.query(
    vector = v['embedding'],
    namespace=INDEX_NAMESPACE,
    top_k=top_k,
    include_values=False,
    include_metadata=True
  )

  my_list = query_result.matches
  descriptions = [entry['metadata']['description'] for entry in my_list]
  most_common_item = max(set(descriptions), key=descriptions.count)

  if most_common_item == v['description']:
      top_k_success = True
  
  if top_k_success:
    top_k_success_cnt += 1

print(f"top_k success rate: {top_k_success_cnt / (len(test_vectors)) * 100}%")

# 8. Run a load test script to simulate 10 concurrent users querying the index

Locust.io is an open-source load testing tool written in Python. It allows you to define user behaviour with Python code and simulate millions of simultaneous users to bombard a system with traffic to test its resilience under heavy load. The (locustfile.py)[./locustfile.py] script re-uses the logic in step #6 to query pinecone. It has a custom event hook that denotes a failure if the top_k result set does not match the search image description. This script will likely fail with a low error rate but you can increase top_k to get a 100% pass rate. The locust summary includes P50 to P100 response time percentiles and QPS(req/s).

In [ ]:
%%bash
locust -f locustfile.py --headless -u 10 -r 1 --run-time 60s --host https://pinecone.io --only-summary

# 9. TEARDOWN: Delete the index 
# WARNING: This next step will delete the PINECONE_INDEX_NAME index and all data in it. DO NOT RUN THIS UNTIL YOU ARE READY OR MANUALLY REMOVE THE INDEX INSTEAD!!! 

In [3]:
if PINECONE_INDEX_NAME in pinecone.list_indexes():
    pinecone.delete_index(PINECONE_INDEX_NAME)

print(f"{PINECONE_INDEX_NAME} index should not exist in index list: {pinecone.list_indexes()}")

james-williams index should not exist in index list: []
